In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_community.tools import (
    WikipediaQueryRun,
    DuckDuckGoSearchRun,
    ArxivQueryRun
)
from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain.tools import tool

In [ ]:
# Custom calculator tool
@tool
def calculator(expression: str) -> str:
    try:
        result = eval(expression)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error in calculation: {e}"


In [ ]:
def setup_simple_tools():
    tools = [
        WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
        DuckDuckGoSearchRun(),
        ArxivQueryRun(api_wrapper=ArxivAPIWrapper()),
        PythonREPLTool(),
        calculator
    ]
    return tools

In [8]:
def create_simple_agent(enhanced=False):    
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # Get tools
    tools = setup_simple_tools()
    
    # Choose prompt
    prompt = enhanced_prompt if enhanced else basic_prompt
    
    # Create agent
    agent = create_react_agent(llm, tools, prompt)
    
    # Create executor
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        max_iterations=2,
        handle_parsing_errors=True
    )
    
    return agent_executor

In [9]:
# Basic prompt without tool guidance
basic_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

# Enhanced prompt with specific tool guidance
enhanced_prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:

{tools}

IMPORTANT - Tool Selection Guidelines:
🔍 Use 'Wikipedia' for: encyclopedic knowledge, historical facts, biographical information, general concepts
🌐 Use 'DuckDuckGo Search' for: current events, recent news, trending topics, real-time information
📚 Use 'ArXiv' for: scientific research papers, academic studies, AI/ML/technical papers
🐍 Use 'Python_REPL' for: complex programming tasks, data analysis, advanced computations
🧮 Use 'calculator' for: simple mathematical calculations and expressions

Choose the most appropriate tool based on the query type!

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do and which tool fits best
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

In [10]:
def test_queries():    
    test_cases = [
        {
            "query": "Who is Albert Einstein?",
            "expected_tool": "Wikipedia",
            "reason": "Biographical information"
        },
        {
            "query": "What's the latest news about OpenAI?",
            "expected_tool": "DuckDuckGo Search", 
            "reason": "Current events"
        },
        {
            "query": "Find papers about neural networks published in 2024",
            "expected_tool": "ArXiv",
            "reason": "Scientific papers"
        },
        {
            "query": "Calculate 15.7 * 23 + 89.5",
            "expected_tool": "calculator",
            "reason": "Simple math calculation"
        }
    ]
    
    print("TOOL SELECTION COMPARISON")
    print("=" * 50)
    
    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest {i}: {test_case['query']}")
        print(f"Expected tool: {test_case['expected_tool']}")
        print(f"Reason: {test_case['reason']}")
        print("-" * 40)
        
        # Test basic prompt
        print("\n🔸 BASIC PROMPT:")
        try:
            basic_agent = create_simple_agent(enhanced=False)
            basic_result = basic_agent.invoke({"input": test_case['query']})
        except Exception as e:
            print(f"Basic agent error: {e}")
        
        print("\n🔹 ENHANCED PROMPT:")
        try:
            enhanced_agent = create_simple_agent(enhanced=True)
            enhanced_result = enhanced_agent.invoke({"input": test_case['query']})
        except Exception as e:
            print(f"Enhanced agent error: {e}")
        
        print("\n" + "=" * 50)

In [12]:
# Example usage
if __name__ == "__main__":
    print("LangChain Agent Tool Selection Demo")
    print("Make sure to set OPENAI_API_KEY environment variable!")
    
    try:
        agent = create_simple_agent(enhanced=True)
        result = agent.invoke({"input": query})
        print(f"\nResult: {result['output']}")
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have set your OPENAI_API_KEY!")

LangChain Agent Tool Selection Demo
Make sure to set OPENAI_API_KEY environment variable!
Error: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)
Make sure you have set your OPENAI_API_KEY!
